In [ ]:
!pip install openai  # OpenAI API for prompting GPT-4o

In [ ]:
import openpyxl
from openai import OpenAI
import os
import json
from tqdm import tqdm
import pandas as pd
import google.generativeai as genai

/usr/local/lib/python3.12/dist-packages/google/colab/_import_hooks/_hook_injector.py:55: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  loader.exec_module(module)


In [ ]:
# Import the API Key (insert key here)
GOOGLE_API_KEY=''
genai.configure(api_key=GOOGLE_API_KEY)

# Reading G-Drive Sheet

In [ ]:
!pip install --quiet gspread oauth2client google-auth
import gspread
from google.colab import auth
import pandas as pd
import google.auth

auth.authenticate_user()

# Authorize gspread client
# Use google.auth.default() to get credentials
credentials, project = google.auth.default()
gc = gspread.authorize(credentials)

def load_sheet(sheet_id, sheet_name="Sheet1"):
    # Load google sheet as dataframe
    worksheet = gc.open_by_key(sheet_id).worksheet(sheet_name)
    data = worksheet.get_all_values()
    df = pd.DataFrame(data[1:], columns=data[0])
    return df

path = "1WZlahRsgIBNi4l23SazRzBZ6K5xM_2j_eeTWzhEVsvE"
sheet = load_sheet(path,"roseWangResponses_mapped_20251112")  # There is only one sheet, so no ambiguity here.
sheet["GPT Rationale"] = ""

Get all the responses and scores

In [ ]:
INPUT_COLUMN = "response" # Specify the column from which input is read (A->1, B->2, etc.)
sheet = sheet[(sheet["title"]=="Asking Questions to Guide Thinking") & (sheet["prompt"].str.startswith(("3","11")))]
sheet = sheet.reset_index(drop=True)
row_count = len(sheet) # Get number of rows. If number of rows to read is already known (or sheet does not terminate where data terminates), place number here (+1) instead.inputs = [sheet.loc[i, INPUT_COLUMN]  for i in range(2, row_count)]  # Column number is set here. YOUR INPUT COLUMN NUMBER GOES HERE.
inputs = [sheet.loc[i, INPUT_COLUMN] for i in range(row_count)]
inputs = [x for x in inputs if x is not None]
print(inputs)

['It doesnt give him the answer but encourages him to think about possible solutions', 'it encourages her to think out why it worked so that she can feel more confident.', 'So the student knows there is something wrong with the previous steps and can find out what they missed from there.', 'It makes sure that she really know how to use the formula', 'It guides them while they think', 'It reenforces their understanding', 'Because it will help them reinforce their understanding', 'Because it will give them a start to build off of', 'It helps them think about what they did', 'Since it is unclear if she knows the difference between width and length as the numbers are the same, I can give her another area that ensures she understands.', 'This strategy will help them confirm or reject their current strategy in solving the problem.', 'It would prompt Matthias to think of a different solution.', 'it indicated his step might not be correct and gives him another opportunity to think it through',

# Prompt Setup



Scoring Prompt Input

In [ ]:
GEMINI_SYSTEM_PROMPT = """
You are a tutor evaluator. Please score the following tutor response to a tutor training scenario involving a middle school student:
-if the tutor's response demonstrates that the tutor understands the importance of encouraging the student to reflect or think critically, score with a 1. Examples of responses scoring a 1 are: “This question helps Tvisha apply her reasoning to a new situation, reinforcing her understanding instead of just memorizing a formula.”; “This will best guide the student's thinking because it will encourage her to think critically about why she took the steps she did.”.
-if the tutor response does NOT demonstrate an understanding of the importance of encouraging the student to reflect or think critically, score with a 0. Sample responses scoring a 0 include: “Suggesting easier problem to solve offers students to think in steps.”; “Make him approach the problem in a correct manner.”.

Once given a response by the user, please return a JSON string following the format, {"Rationale": "your reasoning here", "Score":0/1}
"""

Helper function for response parsing

In [ ]:
def extract_response(response_obj, json=False):
  role = response_obj.choices[0].message.role
  content = response_obj.choices[0].message.content
  if json:
    return {"role": role, "content": content}
  else:
    return (role, content)

## Gemini Call

In [ ]:
# Iterate over all responses
MAX_TOKENS = 1000
TEMPERATURE = 0
RUN_UP_TO = 295 # Sets a maximum index for responses to run. Useful to specify how many responses we want to run on (partial execution). Set to -1 to run them all.
SCORE_COLUMN = "response"  # Change column numbers here to  modify where output is written

MODEL = 'gemini-2.5-pro'
model = genai.GenerativeModel(MODEL, system_instruction=GEMINI_SYSTEM_PROMPT)


if RUN_UP_TO >=  0:  # If an upper bound is set
  inputs_upto = inputs[:RUN_UP_TO]
else:
  inputs_upto = inputs  # Take the whole set of responses
tempScoreList = []
tempDirectionList = []
tempRationaleList = []

for index, inpt in tqdm(enumerate(inputs_upto), total=len(inputs_upto)):
  generation_prompt = "Tutor Response: " + inpt + "\n\n. Your JSON: " # Gemini Change
  generation_config = genai.GenerationConfig(temperature=TEMPERATURE)
  # Use client.chat.completions.create instead of client.generate_text
  gemini_out = model.generate_content(generation_prompt, generation_config=generation_config) # TODO: Potential break point
  print(gemini_out)
  # Extract the content from the response
  content = gemini_out.text.lstrip("```json")[:-4]
  print(content)
  # We now need to parse the JSON into rational and score
  try:
    content_json = json.loads(content)  # Run response through JSON
    score = str(content_json["Score"])  # Cast to string to avoid type inequality
    rationale = str(content_json["Rationale"])  # Fetch the rationale
    sheet.at[index,"Gemini Score"] = score  # Now write both into the dataframe
    sheet.at[index,"Gemini Rationale"] = rationale
    tempScoreList.append(score)
    tempDirectionList.append(generation_prompt)
    tempRationaleList.append(rationale)
    #rationale_cell.value = rationale

  except:
    print("error!")
  #   score_cell.value = "---"
  #   rationale_cell.value = "---"  # Failsafe
  #   if TWO_STAGE:
  #     refined_feedback_cell.value = "---"
print(tempScoreList)
print(tempDirectionList)
print(tempRationaleList)

  0%|          | 1/295 [00:08<39:54,  8.14s/it]

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "```json\n{\n\"Rationale\": \"The tutor's response correctly identifies that the goal is not to provide the answer, but to stimulate the student's own thought process. The phrase 'encourages him to think about possible solutions' directly aligns with the principle of fostering critical thinking and problem-solving skills, which is a key criterion for a score of 1.\",\n\"Score\":1\n}\n```"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 240,
        "candidates_token_count": 85,
        "total_token_count": 998
      },
      "model_version": "gemini-2.5-pro"
    }),
)

{
"Rationale": "The tutor's response c

  1%|          | 2/295 [00:16<38:59,  7.99s/it]

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "```json\n{\n\"Rationale\": \"The tutor's response focuses on encouraging the student to 'think out why it worked.' This directly aligns with the goal of promoting reflection and critical thinking, as the emphasis is on understanding the underlying reasoning behind a successful outcome, rather than just acknowledging the success itself.\",\n\"Score\": 1\n}\n```"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 243,
        "candidates_token_count": 72,
        "total_token_count": 1105
      },
      "model_version": "gemini-2.5-pro"
    }),
)

{
"Rationale": "The tutor's response focuses on encouraging the 

  1%|          | 3/295 [00:23<38:20,  7.88s/it]

...

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "```json\n{\n\"Rationale\": \"This response focuses on the student's cognitive process. By using the phrase 'guide them to realize,' the tutor shows they understand the importance of prompting the student to reflect on their own work and identify their own error, which is a key component of critical thinking.\",\n\"Score\": 1\n}\n```"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 241,
        "candidates_token_count": 72,
        "total_token_count": 1351
      },
      "model_version": "gemini-2.5-pro"
    }),
)

{
"Rationale": "This response focuses on the student's cognitive process. By using the phras

100%|█████████▉| 294/295 [45:47<00:09,  9.37s/it]

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "```json\n{\n\"Rationale\": \"The tutor's response focuses on prompting the student to re-evaluate their own work and thought process (\\\"relook at the question\\\"). The stated goal of \\\"reinforcing his problem-solving\\\" demonstrates an understanding that the objective is to build the student's critical thinking skills, not just to arrive at the correct answer.\",\n\"Score\": 1\n}\n```"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 247,
        "candidates_token_count": 85,
        "total_token_count": 1286
      },
      "model_version": "gemini-2.5-pro"
    }),
)

{
"Rationale": "The tutor's respo

100%|██████████| 295/295 [45:55<00:00,  9.34s/it]

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "```json\n{\n\"Rationale\": \"The tutor's response explicitly mentions that the student will 'reflect on her thought process' and 'create a deeper understanding.' This directly aligns with the scoring criteria for a 1, as it demonstrates a focus on encouraging the student to think critically about their own learning and conceptual understanding.\",\n\"Score\":1\n}\n```"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 250,
        "candidates_token_count": 75,
        "total_token_count": 1086
      },
      "model_version": "gemini-2.5-pro"
    }),
)

{
"Rationale": "The tutor's response explicitly mentions

In [ ]:
for i in range(len(inputs_upto)):
  print(inputs_upto[i])
  print(tempScoreList[i])
  print(tempRationaleList[i])
  print()

It doesnt give him the answer but encourages him to think about possible solutions
1
The tutor's response correctly identifies that the goal is not to provide the answer, but to stimulate the student's own thought process. The phrase 'encourages him to think about possible solutions' directly aligns with the principle of fostering critical thinking and problem-solving skills, which is a key criterion for a score of 1.

it encourages her to think out why it worked so that she can feel more confident.
1
The tutor's response focuses on encouraging the student to 'think out why it worked.' This directly aligns with the goal of promoting reflection and critical thinking, as the emphasis is on understanding the underlying reasoning behind a successful outcome, rather than just acknowledging the success itself.

So the student knows there is something wrong with the previous steps and can find out what they missed from there.
0
This response focuses on the student identifying and correcting a

In [ ]:
print(list(sheet["GPT Score"]))

['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Save G-Drive Sheet

In [ ]:
sheet.to_csv('/content/drive/Shareddrives/PLUS/Research/Clara Stuff/Asking_Questions_to_Guide_thinking_Explain_Gemini_Eval.csv', index=False)